In [70]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
from sklearn import gaussian_process, preprocessing, pipeline

pio.templates.default = "plotly_white"

In [6]:
data = pd.read_csv("data.csv", index_col=0, parse_dates=True)

In [7]:
data

,Update Duration (ms),Reading Date and Time (ISO),Air pressure (mb),Air temperature (C),Tide height (m),Wind direction (deg),Wind gust speed (kn),Wind speed (kn),True air temperature (C),True tide height (m),Independent tide height prediction (m),Independent tide height deviation (m),Dependent tide height prediction (m),Dependent tide height deviation (m),Independent air temperature prediction (C),Independent air temperature deviation (C),Dependent air temperature prediction (C),Dependent air temperature deviation (C)
Update Date and Time (ISO),,,,,,,,,,,,,,,,,,
2007-05-26 12:05:00,0,2007-05-26T12:05:00,NaN,NaN,NaN,NaN,NaN,NaN,16.92,2.30,2.4007,0.039110,2.3715,0.036670,17.2348,0.23222,17.2339,0.23187
2007-05-26 12:10:00,0,2007-05-26T12:10:00,1006.0,16.7,2.3,22.0,12.0,12.4,16.42,2.24,2.4016,0.058707,2.3365,0.045564,17.1932,0.29931,17.1848,0.29732
2007-05-26 12:15:00,0,2007-05-26T12:15:00,1006.0,16.0,2.2,13.0,18.1,13.2,16.00,2.19,2.2945,0.037006,2.2836,0.029853,16.8005,0.25186,16.8130,0.24920
2007-05-26 12:20:00,0,2007-05-26T12:20:00,1006.0,15.9,2.1,16.0,14.2,13.0,15.92,2.14,2.1455,0.021627,2.1553,0.019945,16.0734,0.17207,16.0621,0.17101
2007-05-26 12:25:00,0,2007-05-26T12:25:00,1005.0,16.1,2.1,12.0,14.4,12.0,16.08,2.09,2.0900,0.028609,2.1115,0.023468,15.8401,0.21507,15.8126,0.21158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-05-31 15:55:00,0,2007-05-31T15:55:00,NaN,NaN,NaN,NaN,NaN,NaN,16.00,2.19,2.1759,0.020713,2.2065,0.018579,16.1252,0.15125,16.1222,0.14964
2007-05-31 16:00:00,0,2007-05-31T16:00:00,1007.0,15.9,2.1,123.0,22.3,20.4,15.92,2.08,2.0803,0.019198,2.0998,0.017608,16.0285,0.14422,16.0271,0.14289
2007-05-31 16:05:00,0,2007-05-31T16:05:00,1007.0,15.6,2.0,119.0,21.6,19.8,15.67,1.98,1.9782,0.023006,1.9904,0.020269,15.9775,0.16485,15.9777,0.16287


In [107]:
px.line(data, y=["True air temperature (C)", 'Air temperature (C)'])

In [125]:
model = pipeline.make_pipeline(
    preprocessing.StandardScaler(),
    gaussian_process.GaussianProcessRegressor(
        normalize_y=True, kernel=gaussian_process.kernels.RBF(length_scale_bounds=(0, 1e10))
    ),
)


X, y = (
    data['Air temperature (C)'].dropna().index.astype(int).to_numpy().reshape(-1, 1),
    data['Air temperature (C)'].dropna(),
)

model.fit(X, y)

/Users/theo/Library/Caches/pypoetry/virtualenvs/aims-vYD_rrQz-py3.9/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:357: RuntimeWarning:

divide by zero encountered in log



Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gaussianprocessregressor',
                 GaussianProcessRegressor(kernel=RBF(length_scale=1),
                                          normalize_y=True))])

In [126]:
X_pred = data.index.astype(int).to_numpy().reshape(-1, 1)
y_hat = model.predict(X_pred)
model[1].kernel

RBF(length_scale=1)

In [127]:
data["Air temperature prediction"] = y_hat

In [128]:
px.line(data, y=["Air temperature prediction", "True air temperature (C)", 'Air temperature (C)'])
